In [1]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [ ]:
!pip install pymorphy2
!pip install pymystem3==0.1.10
!pip install rusenttokenize
!pip install razdel

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Задание 1.**

Реализуйте упрощенную версию byte pair encoding (без предварительного разбивания на слова, которое есть в посте о BPE на медиуме - https://towardsdatascience.com/byte-pair-encoding-the-dark-horse-of-modern-nlp-eb36c7df4f10). 

Алгоритм должен работать так:
строки с текстом разбиваются на отдельные символы и далее в цикле из N итераций: а) считаются статистики встречаемости по парам символов и б) топ-K частотных пар склеиваются в один символ

Попробуйте так токенизировать текст с разными параметрами N и K. Проанализируйте словарь уникальных слов для нескольких наборов параметров - сколько уникальных слов получилось, какой токен самый длинный?

(5 баллов)

Чтобы получить 1 бонусный балл - зафиксируйте получившийся словарь и токенизируйте с помощью него текст, который ранее не встречался в корпусе (возьмите рандомную новость из яндекс новостей например). Проанализируйте насколько хорошо токенизировался текст.

In [4]:
news = open('lenta.txt').read()[:200000]

In [5]:
news[:100]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к'

In [6]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return ' '.join(normalized_text)

In [7]:
news_norm = normalize(news)

In [8]:
news_norm[:100]

'бои у сопоцкина и друскеник закончились отступлением германцев неприятель приблизившись с севера к о'

In [9]:
chars = list(news_norm)

In [10]:
print(chars[:5])

['б', 'о', 'и', ' ', 'у']


In [11]:
in_vocab = {}
for i in chars:
  if i in in_vocab.keys():
    in_vocab[i] += 1
  else:
    in_vocab[i] = 1

In [12]:
len(in_vocab)

79

In [13]:
for i in sorted(in_vocab, key = lambda x: in_vocab[x], reverse=True)[:10]:
  print(i, in_vocab[i])

  25514
о 17415
е 13624
и 13200
а 12855
н 10955
т 9841
с 9802
р 8887
в 8013


In [14]:
import copy
def vocab_update(text, n, k):
  vocab = copy.deepcopy(in_vocab)
  cands = {}
  for it in range(n):
    for i in vocab.keys():
      if i != ' ':
        for j in vocab.keys():
          if j != ' ':
            pair = i+j
            if pair in cands.keys():
              continue
            else:
              cands[pair] = text.count(pair)
    winners = sorted(cands, key = lambda x: cands[x], reverse=True)
    cnt = 0
    for w in winners:
      if w in vocab.keys():
        continue
      else:
        vocab[w] = cands[w]
        cnt += 1
      if cnt == k:
        break
  return vocab

In [15]:
test_vocab = vocab_update(news_norm, 30, 5)

In [16]:
len(test_vocab)

229

In [17]:
def bpe_tokenizer(text, vocab):
  #text_norm = normalize(text)
  old_text_tok = list(text)
  new_text_tok = []
  while new_text_tok != old_text_tok:
    if new_text_tok != []:
      old_text_tok = new_text_tok
    i = 0
    new_text_tok = []
    while i <= (len(old_text_tok)-2):
      bi = old_text_tok[i] + old_text_tok[i+1]
      if bi in vocab:
        new_text_tok.append(bi)
        i += 2
      else:
        new_text_tok.append(old_text_tok[i])
        i += 1
      if i == len(old_text_tok)-1:
        new_text_tok.append(old_text_tok[i])

  return new_text_tok

In [18]:
tt = 'Стена была высокая. Рама тоже. А ров был глубоким.'

In [19]:
print(bpe_tokenizer(normalize(tt), test_vocab))

['ст', 'ен', 'а', ' ', 'б', 'ы', 'ла', ' ', 'вы', 'со', 'ка', 'я', ' ', 'ра', 'ма', ' ', 'то', 'же', ' ', 'а', ' ', 'ров', ' ', 'б', 'ы', 'л', ' ', 'г', 'л', 'у', 'бо', 'ки', 'м']


Создадим несколько вариантов словарей и посмотрим, сколько уникальных слов и какой токен самый длинный у каждой комбинации

In [20]:
vocab_lens = []
longest_token = []
for i in [5,10,30,50]: #число итераций
  for j in [3,5,10,20]: #число топовых кандидатов, которых мы добавляем в словарь на каждой итерациии
    v = vocab_update(news_norm, i, j)
    vocab_lens.append(len(v))
    longest = max(v.keys(), key=len)
    longest_token.append([longest, v[longest]])

In [21]:
k = 0
for i in [5,10,30,50]:
  for j in [3,5,10,20]:
    print(i,j,vocab_lens[k], longest_token[k])
    k+=1

5 3 94 ['ст', 2639]
5 5 104 ['ст', 2639]
5 10 129 ['ст', 2639]
5 20 179 ['ени', 775]
10 3 109 ['ст', 2639]
10 5 129 ['ст', 2639]
10 10 179 ['ени', 775]
10 20 279 ['ения', 301]
30 3 169 ['ени', 775]
30 5 229 ['ени', 775]
30 10 379 ['ения', 301]
30 20 679 ['сегодня', 100]
50 3 229 ['ени', 775]
50 5 329 ['ения', 301]
50 10 579 ['росси', 177]
50 20 1079 ['российск', 72]


Видно, чем больше итераций и чем больше мы добавляем кандидатов из топа, тем больше у нас получается словарь и тем длинее в нем появляются токены. Также интересно отметить, что для русских новостей частотным кусочком получилась часть прилагательного "российский"

In [26]:
def params_compare(text, n, k):
  text_norm = normalize(text)
  vocab = vocab_update(news_norm, n, k)
  text_tok = bpe_tokenizer(text_norm, vocab)
  return text_tok

In [29]:
tt1 = params_compare(tt, 5, 3)
tt2 = params_compare(tt, 5, 5)
tt3 = params_compare(tt, 10, 3)
tt4 = params_compare(tt, 10, 5)
tt5 = params_compare(tt, 30, 5)
tt6 = params_compare(tt, 100, 20)

In [30]:
for i in [tt1,tt2,tt3,tt4,tt5,tt6]:
  print(i)

['ст', 'ен', 'а', ' ', 'б', 'ы', 'л', 'а', ' ', 'в', 'ы', 'с', 'о', 'к', 'а', 'я', ' ', 'ра', 'м', 'а', ' ', 'то', 'ж', 'е', ' ', 'а', ' ', 'ро', 'в', ' ', 'б', 'ы', 'л', ' ', 'г', 'л', 'у', 'б', 'о', 'к', 'и', 'м']
['ст', 'ен', 'а', ' ', 'б', 'ы', 'л', 'а', ' ', 'в', 'ы', 'с', 'о', 'к', 'а', 'я', ' ', 'ра', 'м', 'а', ' ', 'то', 'ж', 'е', ' ', 'а', ' ', 'ро', 'в', ' ', 'б', 'ы', 'л', ' ', 'г', 'л', 'у', 'б', 'о', 'к', 'и', 'м']
['ст', 'ен', 'а', ' ', 'б', 'ы', 'л', 'а', ' ', 'в', 'ы', 'с', 'о', 'к', 'а', 'я', ' ', 'ра', 'м', 'а', ' ', 'то', 'ж', 'е', ' ', 'а', ' ', 'ро', 'в', ' ', 'б', 'ы', 'л', ' ', 'г', 'л', 'у', 'б', 'о', 'к', 'и', 'м']
['ст', 'ен', 'а', ' ', 'б', 'ы', 'л', 'а', ' ', 'в', 'ы', 'со', 'ка', 'я', ' ', 'ра', 'м', 'а', ' ', 'то', 'ж', 'е', ' ', 'а', ' ', 'ро', 'в', ' ', 'б', 'ы', 'л', ' ', 'г', 'л', 'у', 'б', 'о', 'к', 'и', 'м']
['ст', 'ен', 'а', ' ', 'б', 'ы', 'ла', ' ', 'вы', 'со', 'ка', 'я', ' ', 'ра', 'ма', ' ', 'то', 'же', ' ', 'а', ' ', 'ров', ' ', 'б', 'ы', 'л', '

Токенизируем часть наших текстов с разными параметрами

In [34]:
for i in [5,10,30]:
  for j in [3,5,10,20]:
    print(f'N = {{}}, K = {{}}, {{}}'.format(i, j, params_compare(news[:100], i, j)))

N = 5, K = 3, ['б', 'о', 'и', ' ', 'у', ' ', 'с', 'о', 'по', 'ц', 'к', 'и', 'на', ' ', 'и', ' ', 'д', 'р', 'у', 'с', 'к', 'ен', 'и', 'к', ' ', 'з', 'а', 'ко', 'н', 'ч', 'и', 'л', 'и', 'с', 'ь', ' ', 'о', 'т', 'ст', 'у', 'п', 'л', 'ен', 'и', 'е', 'м', ' ', 'г', 'е', 'р', 'м', 'ан', 'ц', 'е', 'в', ' ', 'н', 'е', 'пр', 'и', 'я', 'т', 'е', 'л', 'ь', ' ', 'пр', 'и', 'б', 'л', 'и', 'з', 'и', 'в', 'ш', 'и', 'с', 'ь', ' ', 'с', ' ', 'с', 'е', 'в', 'е', 'ра', ' ', 'к']
N = 5, K = 5, ['б', 'о', 'и', ' ', 'у', ' ', 'с', 'о', 'по', 'ц', 'к', 'и', 'на', ' ', 'и', ' ', 'д', 'р', 'у', 'ск', 'ен', 'и', 'к', ' ', 'з', 'а', 'ко', 'н', 'ч', 'и', 'ли', 'с', 'ь', ' ', 'о', 'т', 'ст', 'у', 'п', 'ле', 'ни', 'е', 'м', ' ', 'г', 'ер', 'м', 'ан', 'ц', 'е', 'в', ' ', 'не', 'пр', 'и', 'я', 'те', 'л', 'ь', ' ', 'пр', 'и', 'б', 'ли', 'з', 'и', 'в', 'ш', 'и', 'с', 'ь', ' ', 'с', ' ', 'с', 'е', 'в', 'ер', 'а', ' ', 'к']
N = 5, K = 10, ['б', 'о', 'и', ' ', 'у', ' ', 'со', 'по', 'ц', 'к', 'ин', 'а', ' ', 'и', ' ', 'д',

In [35]:
for i in [[50,10],[100,10]]:
  print(f'N = {{}}, K = {{}}, {{}}'.format(i[0], i[1], params_compare(news[:100], i[0], i[1])))

N = 50, K = 10, ['бо', 'и', ' ', 'у', ' ', 'со', 'по', 'ц', 'ки', 'на', ' ', 'и', ' ', 'др', 'ус', 'ке', 'ник', ' ', 'за', 'кон', 'чи', 'ли', 'сь', ' ', 'от', 'ст', 'уп', 'лени', 'ем', ' ', 'ге', 'рм', 'ан', 'це', 'в', ' ', 'не', 'пр', 'ия', 'тель', ' ', 'пр', 'иб', 'ли', 'зи', 'вш', 'ис', 'ь', ' ', 'с', ' ', 'се', 'ве', 'ра', ' ', 'к']
N = 100, K = 10, ['бо', 'и', ' ', 'у', ' ', 'со', 'по', 'ц', 'ки', 'на', ' ', 'и', ' ', 'др', 'ус', 'ке', 'ник', ' ', 'за', 'кон', 'чи', 'лись', ' ', 'от', 'ступ', 'лени', 'ем', ' ', 'ге', 'рм', 'ан', 'це', 'в', ' ', 'не', 'пр', 'ия', 'тель', ' ', 'пр', 'иб', 'ли', 'зи', 'вш', 'ись', ' ', 'с', ' ', 'се', 'ве', 'ра', ' ', 'к']


При повышении значений параметров выделяются все более крупные кусочки слов, что в принципе является ожидаемым следствием

Попробуем разметить какой-нибудь новый текст

In [38]:
text_norm = normalize(tn)
text_tok = bpe_tokenizer(text_norm, test_vocab) #N = 30 K = 5
print(text_tok)

['но', 'вы', 'й', ' ', 'же', 'ле', 'з', 'но', 'до', 'ро', 'ж', 'ны', 'й', ' ', 'про', 'ек', 'т', ' ', 'ра', 'з', 'у', 'ме', 'ет', 'ся', ' ', 'не', ' ', 'ро', 'ди', 'л', 'ся', ' ', 'из', ' ', 'ни', 'от', 'к', 'уд', 'а', ' ', 'т', 'у', 'р', 'ци', 'я', ' ', 'ст', 'ал', 'а', ' ', 'ин', 'те', 'ре', 'со', 'ва', 'ть', ' ', 'ки', 'та', 'й', 'ск', 'их', ' ', 'в', 'не', 'ш', 'не', 'по', 'ли', 'ти', 'че', 'ск', 'их', ' ', 'ст', 'ра', 'те', 'го', 'в', ' ', 'е', 'щ', 'е', ' ', 'с', ' ', 'x', 'i', 'x', ' ', 'ве', 'ка', ' ', 'та', 'к', ' ', 'ф', 'ил', 'ос', 'о', 'ф', ' ', 'та', 'н', 'ь', ' ', 'с', 'ы', 'т', 'у', 'н', ' ', 'в', ' ', 'с', 'во', 'ей', ' ', 'ра', 'бо', 'те', ' ', '«', ' ', 'у', 'че', 'ние', ' ', 'о', ' ', 'г', 'у', 'ма', 'нн', 'ос', 'ти', ' ', '»', ' ', 'пр', 'из', 'ы', 'ва', 'л', ' ', 'со', 'з', 'да', 'ть', ' ', 'па', 'на', 'з', 'и', 'ат', 'ск', 'ий', ' ', 'со', 'ю', 'з', ' ', 'от', ' ', 'т', 'у', 'р', 'ци', 'и', ' ', 'до', ' ', 'ки', 'та', 'я', ' ', 'а', ' ', 'во', ' ', 'в', 'то', 'ро'

In [39]:
tn = 'Новый железнодорожный проект, разумеется, не родился из ниоткуда. Турция стала интересовать китайских внешнеполитических стратегов еще с XIX века. Так, философ Тань Сытун в своей работе «Учение о гуманности» призывал создать паназиатский союз от Турции до Китая. А во второй половине ХХ века в теории «трех миров» Мао Цзэдуна Турции отводилось важное место в создании пояса безопасности против СССР, главной, по его мнению, гегемонистской силы в мире, отмечает китаист, специалист в области международных отношений Олег Тимофеев.'
print(params_compare(tn, 30, 10))
print(params_compare(tn, 50, 20))

['но', 'вы', 'й', ' ', 'же', 'ле', 'з', 'но', 'до', 'ро', 'ж', 'ны', 'й', ' ', 'пр', 'ое', 'кт', ' ', 'раз', 'у', 'ме', 'ет', 'ся', ' ', 'не', ' ', 'ро', 'ди', 'л', 'ся', ' ', 'из', ' ', 'ни', 'от', 'ку', 'да', ' ', 'ту', 'р', 'ци', 'я', ' ', 'ст', 'ал', 'а', ' ', 'ин', 'те', 'ре', 'со', 'ва', 'ть', ' ', 'ки', 'та', 'йс', 'ки', 'х', ' ', 'вн', 'е', 'ш', 'не', 'по', 'ли', 'ти', 'че', 'ск', 'их', ' ', 'ст', 'ра', 'те', 'го', 'в', ' ', 'е', 'ще', ' ', 'с', ' ', 'x', 'i', 'x', ' ', 'ве', 'ка', ' ', 'та', 'к', ' ', 'ф', 'ил', 'ос', 'о', 'ф', ' ', 'тан', 'ь', ' ', 'с', 'ы', 'ту', 'н', ' ', 'в', ' ', 'св', 'ое', 'й', ' ', 'ра', 'бо', 'те', ' ', '«', ' ', 'у', 'че', 'ние', ' ', 'о', ' ', 'г', 'у', 'ма', 'нн', 'ости', ' ', '»', ' ', 'пр', 'из', 'ыв', 'ал', ' ', 'со', 'з', 'да', 'ть', ' ', 'па', 'на', 'зи', 'ат', 'ск', 'ий', ' ', 'со', 'ю', 'з', ' ', 'от', ' ', 'ту', 'р', 'ции', ' ', 'до', ' ', 'ки', 'та', 'я', ' ', 'а', ' ', 'во', ' ', 'в', 'то', 'ро', 'й', ' ', 'по', 'ло', 'ви', 'не', ' ', 'х'

В принципе, кажется, что результат получается довольно близким к тому, что было при тех же параметрах на тексте из корпуса

**Задание 2.**

Обучите токенизатор из tokenizers на текстовом корпусе. Рассчитайте статистики для idf по корпусу, используя обученный словарь (разбейте корпус на "документы" по новым строкам, каждый "документ" токенизируйте, для каждого слова посчитайте, в скольких документах оно встречается и рассчитайте idf разделив общее количество документов на это число, возьмите логарифм от полученного числа). 
Векторизуйте текст (в мешок слов) аналогично TfidfVectorizer, используя токенизатор и idf статистики (инициализируйте*** пустую матрицу размером (N документов, K слов в словаре) и в цикле по всем документам постепенно заполните ее - токенизируйте документ, рассчитайте TF каждого слова (количество вхождений в документе поделить на общее количество слов в документе), умножьте TF на IDF и, используя индексы слов в словаре, запишите получившееся значение в матрицу)

Формулу для TFIDF можете уточнить тут -  https://ru.wikipedia.org/wiki/TF-IDF

***Чтобы инициализировать разреженную матрицу используйте scipy:
from scipy.sparse import lil_matrix
X = lil_matrix(N, K)

Обучите классификатор на полученных векторах и оцените на кросс-валидации. 

(5 баллов)

In [29]:
!pip install tokenizers

     |████████████████████████████████| 2.9MB 4.3MB/s 


In [40]:
from sklearn.model_selection import cross_val_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [41]:
from tokenizers import CharBPETokenizer, Tokenizer

In [42]:
def tf(document, terms):
    words = normalize(document)
    total_words = len(words)
    doc_counter = Counter(words)
    for word in doc_counter:
      doc_counter[word] /= total_words
    tfs = [0 for _ in range(len(terms))]
    for i in range(len(terms)):
      tfs[i] = doc_counter[terms[i]]
    return tfs

def count_docs_with_word(word, docs):
    counter = 0
    for doc in docs:
        if word in doc:
            counter += 1
    return counter

def idf(documents, terms):
    idfs = [0 for _ in range(len(terms))]
    total_docs = len(documents)
    for i in range(len(terms)):
      docs_with_word = count_docs_with_word(terms[i], documents)
      idf = 1 + math.log((total_docs+1) / (docs_with_word+1))
      idfs[i] = idf
    return idfs

def td_idf(tf, idf, terms):
    return [tf[i] * idf[i] for i in range(len(terms))]

def build_tfidf(corpus, document, idfs, terms):
    doc_tf = tf(document, terms)
    return td_idf(doc_tf, idfs, terms)

In [43]:
import pandas as pd

df = pd.read_csv('dataset_ok.csv')[:30000]
df.head()

,text,label
0,"наебалово века, для долбаёбов\n",INSULT
1,вся дума в таком же положении😁\n,NORMAL
2,а в каком месте массовое столкновение? шрайбик...,NORMAL
3,"значит ли это, что контроль за вывозом крупног...",NORMAL
4,вам не нужен щеночек? очень хорошие 🐶🥰\n,NORMAL


In [44]:
df.label.value_counts()

NORMAL       25440
INSULT        3577
THREAT         685
OBSCENITY      298
Name: label, dtype: int64

In [341]:
#normal_ids = df.index[df['label'] == 'NORMAL'].tolist()

In [343]:
#df = df.drop(normal_ids[:51055]).reset_index(drop=True)

In [345]:
#df.label.value_counts()

NORMAL       10000
INSULT        8584
THREAT        1666
OBSCENITY      682
Name: label, dtype: int64

In [45]:
df['text'].to_csv('corpus.txt', index=None)

In [46]:
tok_sub = CharBPETokenizer()
tok_sub.train('corpus.txt', vocab_size=3000, min_frequency=10,)

In [47]:
print(tok_sub.encode(df.loc[0, 'text']).tokens)

['на', 'еба', 'ло', 'во</w>', 'ве', 'ка</w>', ',</w>', 'для</w>', 'долба', 'ё', 'бов</w>']


In [48]:
len(tok_sub.get_vocab())

3000

In [49]:
vocab = list(tok_sub.get_vocab().keys())

In [50]:
vocab[:5]

['везде</w>', 'счаст', 'ср</w>', '3</w>', '🌆']

In [51]:
from collections import Counter
import operator
import math

In [52]:
texts = df['text']

In [53]:
all_idfs = idf(texts, vocab)

In [54]:
len(vocab), len(all_idfs)

(3000, 3000)

In [55]:
tf_idf_total = []
for document in texts:
  tf_idf_total.append(build_tfidf(texts, document, all_idfs, vocab))

In [56]:
import pandas as pd
frame = pd.DataFrame(tf_idf_total, columns=vocab, index=texts)
frame.head()

,везде</w>,счаст,ср</w>,3</w>,🌆,pro,🐑,ез,в</w>,ху,дра,🐾,людя,🦷</w>,вы,бра,понятно</w>,све,😠,ч</w>,не,🙆,ε</w>,живо,украи,то</w>,власти</w>,писа,посмот,понрави,прекрас,💐</w>,него</w>,🏆,🦺,всю</w>,пишите</w>,банк</w>,стой,💖</w>,...,взять</w>,отнош,чес,✋</w>,⬆</w>,🐓</w>,ди</w>,ение</w>,хаба,дь</w>,гру,зм</w>,настро,‘</w>,ль</w>,россию</w>,раба,убе,все</w>,нул</w>,нь,п</w>,через</w>,🤕,страну</w>,теля,🙄,чере,рам</w>,ние</w>,вания</w>,🐖,🤠,бы,➕,на,этом</w>,⚫,😵,мы
text,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"наебалово века, для долбаёбов\n",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
вся дума в таком же положении😁\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"а в каком месте массовое столкновение? шрайбикус решил набрать лайков? пиши нормальный комментарий, а не эту хрень\n",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"значит ли это, что контроль за вывозом крупногабаритного мусора и работой регионального оператора осуществляют жители домов?\n",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
вам не нужен щеночек? очень хорошие 🐶🥰\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
clf = SGDClassifier(loss="log", max_iter=50)

In [58]:
clf_2 = MultinomialNB(alpha=0.5, fit_prior='bool', class_prior=[1,1,1,1])

In [67]:
clf_3 = SGDClassifier(loss="log", max_iter=50, alpha=0.0001, class_weight='balanced')

In [60]:
X = tf_idf_total

In [61]:
len(X[0])

3000

In [62]:
y = np.array(df.label)

In [63]:
cross_val_score(clf, X, y, scoring="f1_macro")

array([0.23076987, 0.23417476, 0.23567437, 0.23515591, 0.23281145])

In [64]:
cross_val_score(clf_2, X, y, scoring="f1_macro")

array([0.23780051, 0.24171987, 0.23271911, 0.23563234, 0.23939572])

In [68]:
cross_val_score(clf_3, X, y, scoring="f1_macro")

array([0.30919936, 0.33169457, 0.355256  , 0.32147213, 0.30374472])

In [69]:
cross_val_score(clf, X, y, scoring="f1_micro")

array([0.847     , 0.84816667, 0.84816667, 0.84916667, 0.84783333])

In [70]:
cross_val_score(clf_2, X, y, scoring="f1_micro")

array([0.84266667, 0.84166667, 0.84266667, 0.84116667, 0.8405    ])

In [71]:
cross_val_score(clf_3, X, y, scoring="f1_micro")

array([0.81566667, 0.83483333, 0.84133333, 0.82783333, 0.83      ])